<a href="https://colab.research.google.com/github/Vetrivel-PS/JanataHack-Computer-Vision-Hackathon/blob/master/Vgg16_CV_Hack.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install PyDrive

In [2]:
from keras.models import Sequential
from keras.layers import Conv2D
from keras.layers import MaxPooling2D
from keras.layers import Flatten
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import BatchNormalization
import os
import numpy as np
import pandas as pd
from zipfile import ZipFile
from PIL import Image
from keras.preprocessing.image import ImageDataGenerator
from keras.preprocessing import image
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

Using TensorFlow backend.


In [3]:
download = drive.CreateFile({'id': '1VN_c2R2x-Il-YLDec6tmWOTJCFj2mKPB'})
download.GetContentFile('train.csv')
train = pd.read_csv('train.csv')

download = drive.CreateFile({'id': '18eiyECVFkSy3w196yiMilEkgy0Or8o0_'})
download.GetContentFile('test.csv')
test = pd.read_csv('test.csv')

download = drive.CreateFile({'id': '1n4DDHv6Jy5zKxJnUBCi4Jzi4ijH9kyaR'})
download.GetContentFile('sample_submission_yxjOnvz.csv')
sample_submission = pd.read_csv('sample_submission_yxjOnvz.csv')

download = drive.CreateFile({'id': '1tIb3n_N2YebNlTmzmGMnUawy5HdTLKo_'})
download.GetContentFile('train_SOaYf6m.zip')
!unzip train_SOaYf6m.zip -d "cv_dataset"

Archive:  train_SOaYf6m.zip
   creating: cv_dataset/images/
  inflating: cv_dataset/images/0.jpg  
  inflating: cv_dataset/images/1.jpg  
  inflating: cv_dataset/images/10.jpg  
  inflating: cv_dataset/images/100.jpg  
  inflating: cv_dataset/images/1000.jpg  
  inflating: cv_dataset/images/1001.jpg  
  inflating: cv_dataset/images/1002.jpg  
  inflating: cv_dataset/images/1003.jpg  
  inflating: cv_dataset/images/1004.jpg  
  inflating: cv_dataset/images/1005.jpg  
  inflating: cv_dataset/images/1006.jpg  
  inflating: cv_dataset/images/1007.jpg  
  inflating: cv_dataset/images/1008.jpg  
  inflating: cv_dataset/images/1009.jpg  
  inflating: cv_dataset/images/101.jpg  
  inflating: cv_dataset/images/1010.jpg  
  inflating: cv_dataset/images/1011.jpg  
  inflating: cv_dataset/images/1012.jpg  
  inflating: cv_dataset/images/1013.jpg  
  inflating: cv_dataset/images/1014.jpg  
  inflating: cv_dataset/images/1015.jpg  
  inflating: cv_dataset/images/1016.jpg  
  inflating: cv_dataset/im

In [0]:
directory_train_emer='train/emergency'
directory_train_non_emer='train/not_emergency'

directory_vaild_emer='valid/emergency'
directory_valid_non_emer='valid/not_emergency'

parent_directory='cv'

path1=os.path.join(parent_directory, directory_train_emer)
os.makedirs(path1, exist_ok=True)
path1=os.path.join(parent_directory, directory_train_non_emer)
os.makedirs(path1, exist_ok=True)
path1=os.path.join(parent_directory, directory_vaild_emer)
os.makedirs(path1, exist_ok=True)
path1=os.path.join(parent_directory, directory_valid_non_emer)
os.makedirs(path1, exist_ok=True)

In [0]:
#File_Split
dataset_train=pd.read_csv('cv_dataset/train.csv')

path_directory='cv_dataset/images'
for x in dataset_train.query('emergency_or_not==1')['image_names']:
    path2=os.path.join(path_directory, x)
    img=Image.open(path2)
    path_emer=os.path.join(parent_directory, directory_train_emer)
    path_emer=os.path.join(path_emer,x)
    img.save(path_emer)

for x in dataset_train.query('emergency_or_not==0')['image_names']:
    path2=os.path.join(path_directory, x)
    img=Image.open(path2)
    path_emer=os.path.join(parent_directory, directory_train_non_emer)
    path_emer=os.path.join(path_emer,x)
    img.save(path_emer)

In [6]:
#My model is giving 83%

classifier= Sequential()

classifier.add(Conv2D(32,(3,3), padding='same',  input_shape=(224,224,3), activation = "relu"))
classifier.add(BatchNormalization())
classifier.add(MaxPooling2D(pool_size=(2,2))) 

classifier.add(Conv2D(64,(3,3), padding='same', activation = "relu"))
classifier.add(BatchNormalization())
classifier.add(MaxPooling2D(pool_size=(2,2)))
classifier.add(Flatten())

classifier.add(Dense(64, activation = 'relu', kernel_initializer='uniform'))
classifier.add(Dropout(0.2))

classifier.add(Dense(64, activation = 'relu', kernel_initializer='uniform'))
classifier.add(Dropout(0.2))

classifier.add(Dense(1, activation = 'sigmoid', kernel_initializer='uniform'))

classifier.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

from keras.preprocessing.image import ImageDataGenerator
train_datagen = ImageDataGenerator(rescale=1./255, shear_range=0.2, zoom_range=0.2, horizontal_flip=True)

train_set = train_datagen.flow_from_directory('cv/train', 
                                              target_size=(224,224), 
                                              batch_size=32, class_mode='binary')

# test_set = test_datagen.flow_from_directory('drive/My Drive/classification/dataset/test_set', 
#                                             target_size=(180,180), 
#                                             batch_size=32, class_mode='binary')

#filepath='classification_main.hdf5'
#best_model_save=ModelCheckpoint(filepath, monitor='val_accuracy', verbose=1, save_best_only=True, mode='max')
# monitor=EarlyStopping(monitor='val_accuracy', patience=5, verbose=1)
classifier.fit_generator(train_set, epochs=50)

Found 1646 images belonging to 2 classes.
Epoch 1/50
52/52 [==============================] - 24s 453ms/step - loss: 0.6450 - accuracy: 0.6804
Epoch 2/50
52/52 [==============================] - 20s 387ms/step - loss: 0.4914 - accuracy: 0.7625
Epoch 3/50
52/52 [==============================] - 20s 386ms/step - loss: 0.4480 - accuracy: 0.7892
Epoch 4/50
52/52 [==============================] - 20s 389ms/step - loss: 0.3949 - accuracy: 0.8080
Epoch 5/50
52/52 [==============================] - 20s 388ms/step - loss: 0.3574 - accuracy: 0.8499
Epoch 6/50
52/52 [==============================] - 20s 388ms/step - loss: 0.3316 - accuracy: 0.8524
Epoch 7/50
52/52 [==============================] - 20s 389ms/step - loss: 0.2996 - accuracy: 0.8736
Epoch 8/50
52/52 [==============================] - 20s 387ms/step - loss: 0.2924 - accuracy: 0.8827
Epoch 9/50
52/52 [==============================] - 20s 390ms/step - loss: 0.2418 - accuracy: 0.8998
Epoch 10/50
52/52 [==============================

In [47]:
#Vgg-16 is giving accuracy of 0.9201388889

from keras import applications
from keras.utils.np_utils import to_categorical
import math

img_width, img_height = 224,224
top_model_weights_path = 'classification_model.h5'
epochs = 50
batch_size = 64

#Importing Vgg16 without Top layer 
vgg16 = applications.VGG16(include_top=False, weights='imagenet')
datagen = ImageDataGenerator(rescale=1./255, shear_range=0.2, zoom_range=0.2, horizontal_flip=True) 

train_set = datagen.flow_from_directory( 
       'cv/train', 
        target_size=(img_width, img_height), 
        batch_size=batch_size, 
        class_mode=None, 
        shuffle=False) 
     
nb_train_samples = len(train_set.filenames) 
num_classes = len(train_set.class_indices)  
predict_size_train = int(math.ceil(nb_train_samples / batch_size))
     
train_features = vgg16.predict_generator(train_set, predict_size_train)
np.save('train_features.npy', train_features)

train_set_lables = datagen.flow_from_directory( 
   'cv/train', 
   target_size=(img_width, img_height), 
   batch_size=batch_size, 
   class_mode='binary', 
   shuffle=False) 
 
nb_train_samples = len(train_set_lables.filenames) 
num_classes = len(train_set_lables.class_indices)  
train_data = np.load('train_features.npy')  
train_labels = train_set_lables.classes  
train_labels = to_categorical(train_labels, num_classes=num_classes)

#Building own Top Layer
classifier = Sequential() 
classifier.add(Flatten(input_shape=train_data.shape[1:])) 

classifier.add(Dense(32, activation = 'relu', kernel_initializer='he_uniform')) # he_uniform
classifier.add(Dropout(0.1))

classifier.add(Dense(32, activation = 'relu', kernel_initializer='he_uniform')) # he_uniform
classifier.add(Dropout(0.1))

# classifier.add(Dense(64, activation = 'relu', kernel_initializer='he_uniform')) # he_uniform
# classifier.add(Dropout(0.1))


classifier.add(Dense(num_classes, activation = 'softmax', kernel_initializer='he_uniform')) # he_uniform
classifier.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
classifier.fit(train_data, train_labels, epochs=10, batch_size=batch_size)                   

Found 1646 images belonging to 2 classes.
Found 1646 images belonging to 2 classes.
Epoch 1/10
1646/1646 [==============================] - 0s 207us/step - loss: 0.9897 - accuracy: 0.6227
Epoch 2/10
1646/1646 [==============================] - 0s 102us/step - loss: 0.3896 - accuracy: 0.8348
Epoch 3/10
1646/1646 [==============================] - 0s 105us/step - loss: 0.2505 - accuracy: 0.9010
Epoch 4/10
1646/1646 [==============================] - 0s 121us/step - loss: 0.1825 - accuracy: 0.9344
Epoch 5/10
1646/1646 [==============================] - 0s 112us/step - loss: 0.1303 - accuracy: 0.9599
Epoch 6/10
1646/1646 [==============================] - 0s 113us/step - loss: 0.1106 - accuracy: 0.9611
Epoch 7/10
1646/1646 [==============================] - 0s 105us/step - loss: 0.0669 - accuracy: 0.9775
Epoch 8/10
1646/1646 [==============================] - 0s 109us/step - loss: 0.0520 - accuracy: 0.9848
Epoch 9/10
1646/1646 [==============================] - 0s 119us/step - loss: 0.0507

In [48]:
#Predicting Test Images
# dataset_test=pd.read_csv('drive/My Drive/test_vc2kHdQ.csv')
# dataset_submission=pd.read_csv('drive/My Drive/Analytics_Vidya_cv/sample_submission_yxjOnvz.csv')
path_directory='cv_dataset/images'
dataset_test=test
dataset_submission= sample_submission
main_result=[]
for x in dataset_test['image_names']:
    path2=os.path.join(path_directory, x)
    test_image=image.load_img(path2, target_size=(img_height,img_width))
    test_image=image.img_to_array(test_image)
    test_image=np.expand_dims(test_image, axis=0)
    bt_prediction = vgg16.predict(test_image)
    result=classifier.predict(bt_prediction)
    main_result.append(result[0][0])

dataset_submission['emergency_or_not']=main_result 
dataset_submission['emergency_or_not']=np.where(dataset_submission['emergency_or_not']<1, 0, 1)
dataset_submission.head(20)

,image_names,emergency_or_not
0,1960.jpg,0
1,668.jpg,1
2,2082.jpg,0
3,808.jpg,1
4,1907.jpg,0
5,350.jpg,1
6,1557.jpg,0
7,111.jpg,1
8,1952.jpg,0
9,1466.jpg,1


In [0]:
file_name = "Sample_CNN_45_VGG_adam_10_Epochs_he_uniform_Extra_Dense_32_32_Dropout_0pt1"
dataset_submission.to_csv(file_name+'.csv',header=True, index=False)

from google.colab import files
files.download(file_name+'.csv')